[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/07-DCT/block_DCT_compression.ipynb)

# III... video compression

## Parameters

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
!ln -sf ~/MRVC/src/image_3.py .
import image_3 as image
!ln -sf ~/MRVC/src/image_1.py .
import image_1 as component
!ln -sf ~/MRVC/src/block_DCT.py .
!ln -sf ~/MRVC/src/YCoCg.py .
import YCoCg as color
#!ln -sf ~/MRVC/src/color_DCT.py .
#import color_DCT as color
#!ln -sf ~/MRVC/src/RGB.py .
#import RGB as color
import cv2 # pip install opencv-python
!ln -sf ~/quantization/information.py .
import information
!ln -sf ~/quantization/distortion.py .
import distortion
import os
import pylab
!ln -sf ~/quantization/deadzone_quantizer.py .
import deadzone_quantizer as Q
import math
import block_DCT as DCT

In [ ]:
G = 2 # GOP size

In [ ]:
HOME = os.environ["HOME"]
!~/MRVC/sequences/container/runme.sh
test_image = "/tmp/original_"

In [ ]:
block_y_side = block_x_side = 8

In [ ]:
N_components = 3

In [ ]:
entropy_estimator = "PNG"
if entropy_estimator == "PNG":
    def compute_BPP(_image, filename_prefix):
        BPP = image.write(_image, filename_prefix, 0)*8/_image.size
        return BPP
else:
    def compute_BPP(_image, filename_prefix=''):
        entropy = information.entropy(_image.flatten().astype(np.int16))
        return entropy

## Quantization steps

In [ ]:
Q_steps = [128, 64, 32, 16, 8]

## Using same $\Delta$ for all coefficients

In [ ]:
    RD_points_no_RDO = []
for i in range(G):
    x = image.read(test_image, i)
    xx = color.from_RGB(x.astype(np.int16) - 128)

    for Q_step in Q_steps:
        yy = DCT.analyze_image(xx, block_y_side, block_x_side)
        yy_k = DCT.uniform_quantize(yy, block_y_side, block_x_side, N_components, Q_step)
        yy_dQ = DCT.uniform_dequantize(yy_k, block_y_side, block_x_side, N_components, Q_step)
        zz_dQ = DCT.synthesize_image(yy_dQ, block_y_side, block_x_side)
        z_dQ = color.to_RGB(zz_dQ) + 128
        MSE = distortion.MSE(x, z_dQ)
        yy_k_subbands = DCT.get_subbands(yy_k, block_y_side, block_x_side)
        BPP = compute_BPP((yy_k_subbands + 128).astype(np.uint8), f"/tmp/{Q_step}_")
        RD_points_no_RDO.append((BPP, MSE))
        print(i, Q_step, end=' ', flush=True)

In [ ]:
RD_points_no_RDO

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points_no_RDO), label="No RDO")
pylab.title("")
pylab.xlabel("BPP")
pylab.ylabel("MSE")
plt.legend(loc="best")
pylab.show()

## Using RDO (Rate/Distortion Optimization)

### Find the optimal progression of quantization steps

In [ ]:
RD_points = []
RD_slopes = []
N_components = xx.shape[2]
single_list = []

for i in range(G):
    x = image.read(test_image, i)
    blocks_in_y = x.shape[0]//block_y_side
    blocks_in_x = x.shape[1]//block_x_side

    #xx = color.from_RGB(x.astype(np.int16) - 128)
    xx = color.from_RGB(x.astype(np.int16))
    xx[...,0] -= np.average(xx[...,0]).astype(np.int16)
    xx[...,1] -= np.average(xx[...,1]).astype(np.int16)
    xx[...,2] -= np.average(xx[...,2]).astype(np.int16)

    yy = DCT.analyze_image(xx, block_y_side, block_x_side)
    yy = DCT.get_subbands(yy, block_y_side, block_x_side)

    for _y in range(block_y_side):
        for _x in range(block_x_side):
            for _c in range(N_components):
                sbc = yy[blocks_in_y*_y : blocks_in_y*(_y + 1),
                         blocks_in_x*_x : blocks_in_x*(_x + 1),
                         _c]
                sbc_energy = information.average_energy(sbc)
                # The first point of each RD curve has a maximum distortion equal
                # to the energy of the subband and a rate = 0
                RD_points.append([(0, sbc_energy)])
                RD_slopes.append([])

    for _y in range(block_y_side):
        for _x in range(block_x_side):
            for _c in range(N_components):
                sbc = yy[blocks_in_y*_y : blocks_in_y*(_y + 1),
                         blocks_in_x*_x : blocks_in_x*(_x + 1),
                        _c]
                counter = 0
                for Q_step in Q_steps:
                    sbc_k = Q.quantize(sbc, Q_step)
                    sbc_dQ = Q.dequantize(sbc_k, Q_step)
                    MSE = distortion.MSE(sbc, sbc_dQ)
                    BPP = component.write(sbc_k.astype(np.uint8), f"/tmp/{_y}_{_x}_{Q_step}_", 0)*8/xx.size
                    #BPP_Q_indexes = information.PNG_BPP((Q_indexes.astype(np.int32) + 32768).astype(np.uint16), "/tmp/BPP_")[0]
                    #BPP_Q_indexes = information.entropy(Q_indexes.astype(np.int16).flatten())
                    point = (BPP, MSE)
                    RD_points[(_y * block_x_side * N_components + _x * N_components ) + _c].append(point)
                    print("Q_step =", Q_step, "BPP =", point[0], "MSE =", point[1])
                    delta_BPP = BPP - RD_points[(_y * block_x_side * N_components + _x * N_components ) + _c][counter][0]
                    delta_MSE = RD_points[(_y * block_x_side * N_components + _x * N_components ) + _c][counter][1] - MSE
                    if delta_BPP > 0:
                        slope = delta_MSE/delta_BPP
                        RD_slopes[(_y * block_x_side * N_components + _x * N_components) + _c].append((slope, (_y, _x, _c), Q_step))
                    else:
                        slope = 0
                    #RD_slopes[(_y * block_x_side * N_components + _x * N_components) + _c].append((Q_step, slope, (_y, _x, _c)))
                    counter += 1

    def filter_slopes(slopes):
        filtered_slopes = []
        slopes_iterator = iter(slopes)
        prev = next(slopes_iterator)
        for curr in slopes_iterator:
            if prev[0] < curr[0]:
                print(f"deleted {prev}")
            else:
                filtered_slopes.append(prev)
            prev = curr
        filtered_slopes.append(prev)
        return filtered_slopes

    filtered_slopes = []
    for i in RD_slopes:
        filtered_slopes.append(filter_slopes(i))

    for l in filtered_slopes:
        #l = filter_slopes(l)
        for i in l:
            #if i[1] > 0:
            single_list.append(i)

In [ ]:
sorted_slopes = sorted(single_list, key=lambda x: x[0])[::-1]

## Build the optimal RD curve

In [ ]:
optimal_RD_points = []
yy_prog = np.zeros_like(yy)
Q_steps_combination = np.full(shape=(block_x_side, block_y_side, N_components), fill_value=99999999)
for s in sorted_slopes:
    sbc_index = s[1]
    _y = sbc_index[0]
    _x = sbc_index[1]
    _c = sbc_index[2]
    Q_steps_combination[_y, _x, _c] = s[2]
    yy_prog[blocks_in_y*_y : blocks_in_y*(_y + 1),
            blocks_in_x*_x : blocks_in_x*(_x + 1),
            _c] = yy[blocks_in_y*_y : blocks_in_y*(_y + 1),
                     blocks_in_x*_x : blocks_in_x*(_x + 1),
                     _c]
    yy_prog_k = DCT.quantize(yy_prog, Q_steps_combination)
    yy_prog_dQ = DCT.dequantize(yy_prog_k, Q_steps_combination)
    
    MSE = distortion.MSE(yy, yy_prog_dQ)

    BPP = image.write((yy_prog_k + 128).astype(np.uint8), f"/tmp/{_y}_{_x}_{_c}_{s[0]}_", 0)*8/xx.size
    point = (BPP, MSE)
    print("sbc =", sbc_index, "Q_step =", s[2], "BPP =", BPP, "MSE =", MSE)
    optimal_RD_points.append(point)

## Compare

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points_no_RDO), label="No RDO")
pylab.plot(*zip(*optimal_RD_points), label="Using RDO")
pylab.title("Effect of using RDO")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc="best")
#pylab.yscale('log')
#pylab.xscale('log')
pylab.show()